In [1]:
import pandas as pd
import sqlite3 as lite
from NNClassifier import Translator
import os.path
import numpy as np

In [2]:
#local database:
database = 'googDrugData.db'

We start by preparing some training data. We have a (small) database of properly classifed drugs, `df_uses`, and the (much larger) database `df_goog` of drugs labeled by strings of *drug class information* which we've scraped from google. We want to train a neural network to read the strings and output the correct drug classes. So we need to align these databases to get training data...

First, we download a database which connects drugs with their uses:

In [3]:
if os.path.isfile('drug_uses.csv'):
    df_uses = pd.read_csv('drug_uses.csv',index_col=0)
else:
    df_uses = pd.read_csv('https://query.data.world/s/81zz5jv9bpf4n70eoaef1lvjw',index_col=0)
df_uses

,Unnamed: 0,drugname_brand,drugname_generic,anatomical,therapeutic,pharmacologic,chemical,substance,name
0,0,8-MOP,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,PSORALENS FOR TOPICAL USE,METHOXSALEN,METHOXSALEN
1,1,8-MOP,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR SYSTEMIC USE,PSORALENS FOR SYSTEMIC USE,METHOXSALEN,METHOXSALEN
2,2,OXSORALEN,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,PSORALENS FOR TOPICAL USE,METHOXSALEN,METHOXSALEN
3,3,OXSORALEN,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR SYSTEMIC USE,PSORALENS FOR SYSTEMIC USE,METHOXSALEN,METHOXSALEN
4,4,ABILIFY,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE
5,5,ABILIFY,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE CAVOXIL
6,6,ABILIFY,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE LAUROXIL
7,7,ABILIFY,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE HYDRATE
8,8,ABILIFY DISCMELT,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE
9,9,ABILIFY DISCMELT,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE CAVOXIL


Next, we correlate this with our local database of drug uses scraped from the internet. Let's load that database:

In [4]:
with lite.connect(database) as con:
    df_goog = pd.read_sql('select Generic_Name, Drug_Class_Data_clean from goog_drug_class_data;',con)
#Do a bit of cleaning:
df_goog['Generic_Name'] = df_goog['Generic_Name'].apply(lambda x: x.strip())
df_goog

,Generic_Name,Drug_Class_Data_clean
0,SULFACETAMIDE SODIUM,\n\n sulfa antibiotics\n \n\n
1,SULFACETAMIDE SODIUM,\n\n Other\n \n\n sulfa antibiotics\n \...
2,SULFACETAMIDE SODIUM,\n\n sulfa antibiotics\n \n\n
3,SULFACETAMIDE SODIUM,\n\n Other\n \n\n sulfa antibiotics\n \...
4,SULFACETAMIDE SODIUM,\n\n sulfa antibiotic / corticosteroid combin...
5,SULFACETAMIDE SODIUM,\n\n Other\n \n\n sulfa antibiotic / cort...
6,SULFACETAMIDE SODIUM,\n\n sulfa antibiotics\n \n\n
7,SULFACETAMIDE SODIUM,\n\n Other\n \n\n sulfa antibiotics\n \...
8,SULFACETAMIDE SODIUM,\n\n Sulfonamide Antibacterial [EPC]\n \n
9,SULFACETAMIDE SODIUM,\n\n Sulfonamides [Chemical/Ingredient]\n \n


In [5]:
df = pd.merge(df_uses[['drugname_generic','anatomical','therapeutic','pharmacologic']]
                      ,df_goog,
              left_on='drugname_generic',
              right_on='Generic_Name',
              how='inner').drop('Generic_Name',1).drop_duplicates(keep='first').reset_index(drop=True)
df

,drugname_generic,anatomical,therapeutic,pharmacologic,Drug_Class_Data_clean
0,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n psoralens\n \n\n
1,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n Other\n \n\n psoralens\n \n\n\n\n\...
2,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,"\n\n Antipsoriatic\n \n ,\n \n Hypopigmentat..."
3,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n Photoabsorption [moa]\n \n\n
4,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n photoactivated Radical Generator [epc]\n...
5,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n photosensitizing Activity [pe]\n \n\n
6,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n psoralen [epc]\n \n\n
7,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n psoralens [chemical/ingredient]\n \n\n
8,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n Photoabsorption [MoA]\n \n
9,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n Photoactivated Radical Generator [EPC]\n \n


Now let's prepare a list of vocabulary for the drug reference phrases we found via google, and a dictionary between vocabulary and words...

In [12]:
translatorObj = Translator(input_phrases = df['Drug_Class_Data_clean'],input_targets = df['therapeutic'])

Most common words are: 
[('<eof>', 9969), (',', 1425), ('-', 1423), ('sodium', 1305), ('fluorure', 1290), (')', 997), ('(', 997), ('other', 877), ('medicine', 665), ('agent', 653)]
We dropped 1245 uninformative training entries, 12.489% of the input set


Save the Translator...

In [13]:
translatorObj.save('training_data.npz')

Translate the unknown drugs to be processed by the RNN...

In [14]:
df_goog['Drug_Class_Data_clean'] = df_goog['Drug_Class_Data_clean'].apply(translatorObj.clean_phrase)
df_goog['Drug_Class_Data_ids'] = df_goog['Drug_Class_Data_clean'].apply(translatorObj.translate_phrase)
df_goog



,Generic_Name,Drug_Class_Data_clean,Drug_Class_Data_ids
0,SULFACETAMIDE SODIUM,sulfa antibiotics eof <eof>,"[1439, 818, 598, 598, 1143, 1143, 1143, 1143, ..."
1,SULFACETAMIDE SODIUM,other sulfa antibiotics accutane aczone atrali...,"[1314, 1439, 818, 273, 409, 1400, 778, 815, 77..."
2,SULFACETAMIDE SODIUM,sulfa antibiotics eof <eof>,"[1439, 818, 598, 598, 1143, 1143, 1143, 1143, ..."
3,SULFACETAMIDE SODIUM,other sulfa antibiotics amoxil augmentin bactr...,"[1314, 1439, 818, 682, 945, 1048, 75, 219, 191..."
4,SULFACETAMIDE SODIUM,sulfa antibiotic / corticosteroid combinations...,"[1439, 462, 495, 824, 271, 598, 598, 1143, 114..."
5,SULFACETAMIDE SODIUM,<eof>,"[598, 1143, 1143, 1143, 1143, 1143, 1143, 1143..."
6,SULFACETAMIDE SODIUM,sulfa antibiotics eof <eof>,"[1439, 818, 598, 598, 1143, 1143, 1143, 1143, ..."
7,SULFACETAMIDE SODIUM,other sulfa antibiotics amoxil augmentin bactr...,"[1314, 1439, 818, 682, 945, 1048, 75, 219, 191..."
8,SULFACETAMIDE SODIUM,sulfonamide antibacterial ( epc ) eof <eof>,"[598, 931, 1162, 123, 109, 598, 598, 1143, 114..."
9,SULFACETAMIDE SODIUM,sulfonamides ( chemical / ingredient ) eof <eof>,"[1055, 1162, 141, 495, 376, 109, 598, 598, 114..."


save the results...

In [15]:
with open('data_to_process.npz','wb') as file:
    np.savez(file,columns = df_goog.columns, data = df_goog.as_matrix())

Check that it loads properly:

In [16]:
with open('data_to_process.npz','rb') as file:
    npzfile = np.load(file)
    df_test = pd.DataFrame(columns = npzfile['columns'],data=npzfile['data'])

In [17]:
df_test

,Generic_Name,Drug_Class_Data_clean,Drug_Class_Data_ids
0,SULFACETAMIDE SODIUM,sulfa antibiotics eof <eof>,"[1439, 818, 598, 598, 1143, 1143, 1143, 1143, ..."
1,SULFACETAMIDE SODIUM,other sulfa antibiotics accutane aczone atrali...,"[1314, 1439, 818, 273, 409, 1400, 778, 815, 77..."
2,SULFACETAMIDE SODIUM,sulfa antibiotics eof <eof>,"[1439, 818, 598, 598, 1143, 1143, 1143, 1143, ..."
3,SULFACETAMIDE SODIUM,other sulfa antibiotics amoxil augmentin bactr...,"[1314, 1439, 818, 682, 945, 1048, 75, 219, 191..."
4,SULFACETAMIDE SODIUM,sulfa antibiotic / corticosteroid combinations...,"[1439, 462, 495, 824, 271, 598, 598, 1143, 114..."
5,SULFACETAMIDE SODIUM,<eof>,"[598, 1143, 1143, 1143, 1143, 1143, 1143, 1143..."
6,SULFACETAMIDE SODIUM,sulfa antibiotics eof <eof>,"[1439, 818, 598, 598, 1143, 1143, 1143, 1143, ..."
7,SULFACETAMIDE SODIUM,other sulfa antibiotics amoxil augmentin bactr...,"[1314, 1439, 818, 682, 945, 1048, 75, 219, 191..."
8,SULFACETAMIDE SODIUM,sulfonamide antibacterial ( epc ) eof <eof>,"[598, 931, 1162, 123, 109, 598, 598, 1143, 114..."
9,SULFACETAMIDE SODIUM,sulfonamides ( chemical / ingredient ) eof <eof>,"[1055, 1162, 141, 495, 376, 109, 598, 598, 114..."
